In [1]:
import os
HOME = os.getcwd()
print(HOME)

/content


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os
HOME = os.getcwd()
print(HOME)
!mkdir {HOME}/datasets
%cd {HOME}/datasets
!mkdir train_data

!cp -r /content/drive/MyDrive/MVA2023_Challenge/data/mva2023_sod4bird_train /content/datasets/train_data


!pwd

%cd /content/datasets/train_data/mva2023_sod4bird_train



!ls

!unzip -q images.zip -d/content/datasets/train_data/mva2023_sod4bird_train/

/content
/content/datasets
/content/datasets
/content/datasets/train_data/mva2023_sod4bird_train
annotations  images.zip  LICENSE


In [4]:
import torch
!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0
torch:  2.2 ; cuda:  cu121


In [5]:
import os
import torch
import torch.utils.data
import torchvision
from PIL import Image
from pycocotools.coco import COCO
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy as np
import cv2

In [6]:
class myOwnDataset(torch.utils.data.Dataset):
    def __init__(self, root, annotation, transforms=None):
        self.root = root
        self.transforms = transforms
        self.coco = COCO(annotation)
        self.ids = list(sorted(self.coco.imgs.keys()))

    def __getitem__(self, index):
        # Own coco file
        coco = self.coco
        # Image ID
        # print("index is", index)
        img_id = self.ids[index]
        # print("image id is",img_id)
        # List: get annotation id from coco
        ann_ids = coco.getAnnIds(imgIds=img_id)
        # print("ann_ids", ann_ids)
        # Dictionary: target coco_annotation file for an image
        coco_annotation = coco.loadAnns(ann_ids)
        # print(coco_annotation)
        # path for input image
        path = coco.loadImgs(img_id)[0]['file_name']
        # print("path is", path)
        # open the input image
        img = Image.open(os.path.join(self.root, path))

        # number of objects in the image
        num_objs = len(coco_annotation)
        # print(num_objs)
        # Bounding boxes for objects
        # In coco format, bbox = [xmin, ymin, width, height]
        # In pytorch, the input should be [xmin, ymin, xmax, ymax]
        # print("tests for ovo",coco_annotation)
        boxes = []
        for i in range(num_objs):
            # print("i is",i)
            # print(coco_annotation[i]['bbox'])

            xmin = coco_annotation[i]['bbox'][0]
            ymin = coco_annotation[i]['bbox'][1]
            width = coco_annotation[i]['bbox'][2]
            height = coco_annotation[i]['bbox'][3]
            boxes.append([xmin, ymin, width, height])
            # Inside the loop where bounding boxes are plotted

        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        # Labels (In my case, I only one class: target class or background)
        labels = torch.ones((num_objs,), dtype=torch.int64)
        # Tensorise img_id
        img_id = torch.tensor([img_id])
        # Size of bbox (Rectangular)
        areas = []
        for i in range(num_objs):
            areas.append(coco_annotation[i]['area'])
        areas = torch.as_tensor(areas, dtype=torch.float32)
        # Iscrowd
        iscrowd = torch.zeros((num_objs,), dtype=torch.int64)

        # Annotation is in dictionary format
        my_annotation = {}
        my_annotation["boxes"] = boxes
        my_annotation["labels"] = labels
        my_annotation["image_id"] = img_id
        my_annotation["area"] = areas
        my_annotation["iscrowd"] = iscrowd


        if self.transforms is not None:
            img = self.transforms(img)

        return img, my_annotation

    def __len__(self):
        return len(self.ids)

In [7]:
def get_transform():
    custom_transforms = []
    custom_transforms.append(torchvision.transforms.Resize((896, 1600)))
    custom_transforms.append(torchvision.transforms.ToTensor())
    return torchvision.transforms.Compose(custom_transforms)

In [8]:
# path to your own data and coco file
# mva2023_sod4bird_train\images
train_data_dir = '/content/datasets/train_data/mva2023_sod4bird_train/images'
# train_data_dir = r'C:\chandan\mva2023_sod4bird_train\images'
# train_coco = '/content/drive/MyDrive/DL_Annotated_Dataset/Annotations/New_json_updated/updated_train_data.json'
train_coco= '/content/datasets/train_data/mva2023_sod4bird_train/annotations/split_train_coco.json'
val_data_dir = '/content/datasets/train_data/mva2023_sod4bird_train/images'
# val_coco = '/content/drive/MyDrive/DL_Annotated_Dataset/Annotations/New_json_updated/updated_val_data.json'
val_coco = '/content/datasets/train_data/mva2023_sod4bird_train/annotations/split_val_coco.json'

# create own Dataset
my_dataset_train = myOwnDataset(root=train_data_dir,
                          annotation=train_coco,
                          transforms=get_transform()
                          )
# create own Dataset
my_dataset_val = myOwnDataset(root=val_data_dir,
                          annotation=val_coco,
                          transforms=get_transform()
                          )

# collate_fn needs for batch
def collate_fn(batch):
    return tuple(zip(*batch))

# Batch size
train_batch_size = 1

# own DataLoader
data_loader_train = torch.utils.data.DataLoader(my_dataset_train,
                                          batch_size=train_batch_size,
                                          shuffle=True,
                                          num_workers=0,
                                          collate_fn=collate_fn)
# own DataLoader
data_loader_val = torch.utils.data.DataLoader(my_dataset_val,
                                          batch_size=train_batch_size,
                                          shuffle=True,
                                          num_workers=0,
                                          collate_fn=collate_fn)

loading annotations into memory...
Done (t=0.25s)
creating index...
index created!
loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


In [9]:
!pip install einops wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 80.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 28.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 277.3/277.3 kB 35.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 9.5 MB/s eta 0:00:00


In [10]:
# Libraries
import torch
from torch import nn, einsum
import torch.nn.functional as F
import torch.optim as optim
import cv2
import numpy as np
import matplotlib.pyplot as plt
import einops
from einops import rearrange
import tqdm
import wandb

In [11]:
wandb.login(key="04a9fd20364b3eef92aa289cdeef1128e8d87924")
wandb.init(project='Final-Proj', name='Our-without-dataAug')

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: chandan-yadav (my_dl_assignments). Use `wandb login --relogin` to force relogin


In [12]:
import sys
sys.path.append('/content/drive/MyDrive/Main')

In [13]:
# Custom Libraries
from backbone import Backbone
from neck_bak import swin_t_neck
from neck import Neck
from swin_fpn_neck import SwinFPNNeck
from head import Head
# from model import Model

In [14]:
# Generating ground truth

def calculate_offset(image_size, bbox):
    x_center = (bbox[0] + bbox[2]) / 2
    y_center = (bbox[1] + bbox[3]) / 2
    offset_x = x_center / image_size[1]
    offset_y = y_center / image_size[0]
    return offset_x, offset_y

def calculate_width_height(bbox):
    width = bbox[2]
    height = bbox[3]
    return width, height

def generate_ground_truth(image_size, bboxs):
    heatmap = np.zeros(image_size)
    widthmap = np.zeros(image_size)
    heightmap = np.zeros(image_size)
    offsetmap = np.zeros((2,image_size[0], image_size[1]))
    for bbox in bboxs:
        x_center = int((bbox[0] + bbox[2]) / 2)
        y_center = int((bbox[1] + bbox[3]) / 2)
        heatmap[x_center, y_center] = 1
        width, height = calculate_width_height(bbox) # Set the center of the bounding box to 1
        widthmap[x_center, y_center] = width
        heightmap[x_center, y_center] =  height
        offset = calculate_offset(image_size, bbox)
        offsetmap[0,x_center, y_center] = offset[0]
        offsetmap[1,x_center, y_center] = offset[1]

    return [torch.tensor(x) for x in [heatmap, widthmap, heightmap, offsetmap]]

In [15]:
def upscale_predictions(preds: list[torch.tensor], tgt_size: np.ndarray, intensity_thresh: float=0.8):
    heat, w, h, o = preds

    heat = heat.cpu().detach().numpy()
    heat = np.where(heat > intensity_thresh, heat, 0)
    heat = torch.tensor(heat)
    heat = F.interpolate(heat.float(), size=tgt_size, mode='bilinear', align_corners=False)

    w = F.interpolate(w.float(), size=tgt_size, mode='bilinear', align_corners=False)

    h = F.interpolate(h.float(), size=tgt_size, mode='bilinear', align_corners=False)

    o = F.interpolate(o.float(), size=tgt_size, mode='bilinear', align_corners=False)
    # o[1] = f.interpolate(o[1].float(), size=tgt_size, mode='bilinear', align_corners=False)


    heat = einops.rearrange(heat, 'b c h w -> h (w c b)')
    heat = heat.detach().numpy()
    w = einops.rearrange(w, 'b c h w -> h (w c b)')
    w = w.cpu().detach().numpy()
    h = einops.rearrange(h, 'b c h w -> h (w c b)')
    h = h.cpu().detach().numpy()
    o = einops.rearrange(o, 'b c h w -> (b c) h w')
    o = o.cpu().detach().numpy()


    heat = np.where(heat==0, heat, 0)
    w = np.where(heat==0, w, 0)
    h = np.where(heat==0, h, 0)
    o = np.where(heat==0, o, 0)
    # o[1] = np.where(heat==0, o[1], 0

    return [torch.tensor(x) for x in [heat, w, h, o]]


In [16]:
class CustomLoss(nn.Module):
    def __init__(self):
        super(CustomLoss, self).__init__()
        self.heatmap_loss = nn.BCEWithLogitsLoss()
        self.sigmoid = nn.Sigmoid()
        self.l1_loss = nn.L1Loss()

    def forward(self, preds, truth):
      p_heat, p_w, p_h, p_o = preds
      g_heat, g_w, g_h, g_o = truth
      heatmap_loss = self.heatmap_loss(self.sigmoid(p_heat), self.sigmoid(g_heat))
      width_loss = self.l1_loss(p_w, g_w)
      height_loss = self.l1_loss(p_h, g_h)
      offset_loss = self.l1_loss(p_o, g_o)
      return heatmap_loss + width_loss + height_loss + offset_loss


# def loss_fn(preds, truth):
#     p_heat, p_w, p_h, p_o = [torch.tensor(x).float() for x in preds]
#     g_heat, g_w, g_h, g_o = [torch.tensor(x).float() for x in truth]
#     heatmap_loss = F.binary_cross_entropy_with_logits(F.sigmoid(p_heat), F.sigmoid(g_heat))
#     width_loss = F.l1_loss(p_w, g_w)
#     height_loss = F.l1_loss(p_h, g_h)
#     offset_loss = F.l1_loss(p_o, g_o)
#     return heatmap_loss + width_loss + height_loss + offset_loss

In [17]:
backbone = Backbone(hid_dim=96, layers=[2, 2, 2, 2], heads=[3, 6, 12, 24])

In [18]:
neck_t = SwinFPNNeck(hid_dim=96, layers=[2,2,2,2], heads=[24, 12, 6, 3], channels=768)

In [19]:
head = Head(in_channels=96, num_classes=1)

In [20]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [21]:
class Model(torch.nn.Module):
    def __init__(self, backbone, neck, head):
        super(Model, self).__init__()
        self.backbone = backbone
        self.neck = neck
        self.head = head

    def forward(self, x):
        features, feature_maps = self.backbone(x)
        # features = features.permute(0, 2, 3, 1)
        # print(features.permute(0, 2, 3, 1).shape)
        features = self.neck(features.permute(0, 2, 3, 1), feature_maps)
        # features = features.permute(0, 3, 1, 2)
        # print("features: ", features.shape)
        detection_output = self.head(features.permute(0, 3, 1, 2))
        # print(detection_output.shape)
        return detection_output

In [22]:
model = Model(backbone, neck_t, head)
model.to(device)
optimizer = optim.SGD(model.parameters(), lr=3e-4)

num_epochs = 10

IMG_SIZE = (2160,3840)

THRESHOLD = 0.8

loss_fn = CustomLoss()

In [ ]:
for epoch in range(num_epochs):
    model.train()
    with tqdm.tqdm(total=len(data_loader_train), desc=f'Epoch {epoch + 1}/{num_epochs}', unit='batch') as pbar:
        losses = []

        for data, bboxes in data_loader_train:
          # data = data.resize()
          data = data[0].unsqueeze(0).float()
          data = data.to(device)
          preds = model(data)
          preds = upscale_predictions(preds, tgt_size=IMG_SIZE, intensity_thresh=THRESHOLD)
          preds = [x.to(device) for x in preds]
          targets = generate_ground_truth(IMG_SIZE, bboxes[0]["boxes"])
          targets = [x.to(device) for x in targets]
          loss = loss_fn(preds,  targets)
          loss.requires_grad = True
          losses.append(loss.item())
          optimizer.zero_grad()
          loss.backward()
          optimizer.step()
          wandb.log({"loss": loss})

        # Update tqdm
        pbar.update(1)
        pbar.set_postfix({'loss': losses[-1]})


        # Log loss to WandB
        # wandb.log({"loss": loss.item()})

print("Training finished!")

wandb.finish()


Epoch 1/10:   0%|          | 0/8880 [00:00<?, ?batch/s]

Streaming output truncated to the last 5000 lines.
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1

Epoch 2/10:   0%|          | 0/8880 [00:00<?, ?batch/s]

Streaming output truncated to the last 5000 lines.
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1

Epoch 3/10:   0%|          | 0/8880 [00:00<?, ?batch/s]

Streaming output truncated to the last 5000 lines.
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1

Epoch 4/10:   0%|          | 0/8880 [00:00<?, ?batch/s]

1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
